In [214]:
## package setup and import
schema.drop(True)
import numpy as np
import datajoint as dj
import pandas as pd
import os

from scipy.io import loadmat
datafile = loadmat('FakeSubject_TestInfoseek_20200730_155906.mat')

folder = os.getcwd()
dj.config['stores'] = {
    'raw_data':{
        'protocol':'file',
        'location': os.path.join(folder,'raw_data')
    }
}

## datajoint setup/import for database management
dj.__version__

dj.config['database.host'] = '127.0.0.1'
dj.config['database.port'] = 3306
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'simple'
dj.config['fetch_format'] = 'array'
dj.config['enable_python_native_blobs'] = True 


connection = dj.conn()
connection

dj.config.save_global()

schema = dj.schema('behavioral')

In [215]:
@schema
class MouseAttributes(dj.Manual): ## Manual entry table for mouse identification/attribute labelling
    definition = """
    mouse_id : int 
    ---
    sex = 'U': enum('F','M','U')
    imaging = null : int
    real_infoseek_data = null : int
    real_infoseek_no_imaging = null : int
    mean_preference = null : varchar(1000)
    full_values = null : int
    one_vs_four = null : int
    one_vs_eight = null : int
    exp_type = null : varchar(1000)
    reward_type = null : varchar(1000)
    comments = null : varchar(1000)
    imaging = null : int
    imaging_comments = null : varchar(2000)
    """
mouse_subject = MouseAttributes()
mouse_subject

# MouseAttributes.insert1({'mouse_id' : 0, 'mean_preference': 'test entry','exp_type': '3 port long'})


mouse_id,sex,imaging,real_infoseek_data,real_infoseek_no_imaging,mean_preference,full_values,one_vs_four,one_vs_eight,exp_type,reward_type,comments,imaging_comments


In [217]:
## need to figure out a way to read 'data' blobs from files -- fixed

## manual entry table with raw data files from each session
@schema
class TestingSession(dj.Manual): ## Manual entry table for each recording session -- want to automate (imported data table - import table characteristics from recording files)
    definition = """
    date : varchar(50)
    time : varchar(50)
    ---
    datafile : varchar(1000)
    session_data : blob@raw_data
    comments = null : varchar(1000)
    init_settings : longblob
    final_settings : longblob
    """


    def insertSession(self,filename):
        data = loadmat(filename)
        date = data['SessionData']['Info'][0][0][0][0][1][0]
        time = data['SessionData']['Info'][0][0][0][0][2][0]
        settings = pd.DataFrame(data['SessionData']['TrialSettings'][0][0][0])
        init_settings = settings.iloc[0]
        final_settings = settings.iloc[len(settings)-1]
        TestingSession.insert1({'datafile':filename,'session_data':data['SessionData'],'date':date,'time':time,'init_settings':init_settings,'final_settings':final_settings},skip_duplicates=True)
test = TestingSession()



test.insertSession('FakeSubject_TestInfoseek_20200730_155906.mat')
test 

date,time,datafile,session_data,comments,init_settings,final_settings
30-Jul-2020,16:02:57,FakeSubject_TestInfoseek_20200730_155906.mat,=BLOB=,None,=BLOB=,=BLOB=


In [218]:

## pull in raw data from TestingSession and create new keys that take and separate session data
## add in other secondary keys when outcome works
@schema 
class BehavioralTable(dj.Imported):
    definition = """
    -> TestingSession
    ---
    outcome : blob@raw_data
    """
    
    def make(self,key):
        print(key)
#         filename = (RecordingSession & key).fetch1()['filename']
        key['outcome'] = (TestingSession & key).fetch1['session_data']
        self.insert(key)

#         data = loadmat(filename)
#         key.update({'data':data})
#         self.insert1(key)

@schema 
class TestTable(dj.Imported):
    definition = """
    -> TestingSession
    ---
    outcome : longblob
    """
    
    def make(self,key):
        print(key)
#         filename = (RecordingSession & key).fetch1()['filename']
        data = (TestingSession & key).fetch1['data']['SessionData']
        outcomedf = pd.DataFrame(data['Outcomes'][0][0][0])
        key['outcome'] = outcomedf
        self.insert(key)








# @schema
# class BehavioralProcessed(dj.Imported):
#     definition = """
#     -> TestingSession
#     ---
#     outcomes : blob@raw_data
#     """
    
#     def make(self,key):
#         print(key)
#         #pull outcomes from data file
#         outcomes = (TestingSession & key).fetch1()['data']['SessionData']['Outcomes'][0][0][0]
#         key.update({'outcomes' : outcomes})
#         self.insert1(key)


In [219]:
## need to find fix
TestTable.populate()

DataJointError: A relation must have primary dependencies for auto-populate to work

In [105]:
## matlab imported data represented as dictionary with numerous arrays inside
## best way to pull out individual values seems to be dataframes for each, input as blobs?
data = loadmat('FakeSubject_TestInfoseek_20200730_155906.mat')
frametest = pd.DataFrame(data['SessionData']['Outcomes'][0][0][0])
frametest
# test.insert1({'session_id': 3,'datafile':'dataframetest','data':frametest})
test1 = test & {'datafile' : 'dataframetest'}
test1.fetch()[0][2]

,0
0,11
1,15
2,16
3,16
4,17
5,17
6,17
7,12
8,12
9,19


In [184]:
datetest = pd.DataFrame(data['SessionData']['TrialSettings'][0][0][0])
datetest


,GUI
0,"[[[[[1000]], [[5]], [[0]], [[3]], [[2]], [[1]]..."
1,"[[[[[1000]], [[5]], [[0]], [[3]], [[2]], [[1]]..."
2,"[[[[[1000]], [[5]], [[0]], [[3]], [[2]], [[1]]..."
3,"[[[[[1000]], [[5]], [[0]], [[3]], [[2]], [[1]]..."
4,"[[[[[1000]], [[5]], [[0]], [[3]], [[2]], [[1]]..."
5,"[[[[[1000]], [[5]], [[0]], [[3]], [[2]], [[1]]..."
6,"[[[[[1000]], [[5]], [[0]], [[3]], [[2]], [[1]]..."
7,"[[[[[1000]], [[5]], [[0]], [[3]], [[2]], [[1]]..."
8,"[[[[[1000]], [[5]], [[0]], [[3]], [[2]], [[1]]..."
9,"[[[[[1000]], [[5]], [[0]], [[3]], [[2]], [[1]]..."
